# MIS
In a graph, the maximal independent set, also known as maximal stable set is an independent set that is not a subset of any other independent set. In other words, it is a set of vertices such that no two vertices in the set are adjacent. In this notebook, we will build a learning-based model to find the maximal independent set in a graph.

<img src="img/Independent_set_graph.png" alt="Solved MIS" style="width:400px; height:400px;">

SeaPearl currently supports learning-based value selection models trained with Reinforcement Learning. To train Reinforcement Learning agents, we start by generating training instances and we let the agent learn a value selection heuristic. Like all other Reinforcement Learning tasks, we need to define a reward function, a state representation and an action space. Finally, we also need to build a neural network that will learn the value selection heuristic. All of these components will be defined in the following sections.

## Setup
We will begin by activating the environment and importing the necessary packages.

In [ ]:
using Revise
using Pkg
Pkg.activate("../../../../")
Pkg.instantiate()
using SeaPearl
using Flux
using LightGraphs
using Random
using BSON: @save, @load
using ReinforcementLearning
using CSV
const RL = ReinforcementLearning

## Generating instances

SeaPearl provides a number of instance generators, including one for the MIS problem. Under the hood, this generator creates Barabasi-Albert graphs with `n` vertices. The graphs are grown by adding new vertices to an initial that has `k` vertices. New vertices are connected by `k` edges to `k` different vertices already present in the system by preferential attachment. The resulting graphs are undirected.

<img src="img/450px-Barabasi_albert_graph.png" alt="Barabasi-Albert Graph" style="width:600px; height:200px;">

In [2]:
numInitialVertices = 3
numNewVertices = 8
instance_generator = SeaPearl.MaximumIndependentSetGenerator(numNewVertices, numInitialVertices)

SeaPearl.MaximumIndependentSetGenerator(8, 3)

## The Reinforcement Learning setup

### The state representation
In SeaPearl, the state $s_t$ is defined as a pair $s_t = (P_t, x_t)$, with $P_t$ a partially solved combinatorial optimization problem and $x_t$ a variable selected at time $t$ of an episode. A terminal episode is reached if all variables are fixed or if a failure is detected.

### The action space
Given a state $s_t = (P_t, x_t)$, an action $a_t$ represents the selection of a value $v$ for the variable $x_t$. The action space is defined as the set of all possible values for the variable $x_t$ at time $t$.

### The transition function
Given a state $s_t = (P_t, x_t)$ and an action $a_t = v$, the transition function is comprised of three steps;
1. The value of variable $x_{267}$ is assigned as $v$ (i.e., $D(x_{t+1}) = v$).
2. The fix-point operation is applied on $P_t$ to prune the domains (i.e., $P_{t+1} = \text{{fixPoint}}(P_t)$).
3. The next variable to branch on is selected (i.e., $x_{t+1} = \text{{nextVariable}}(P_{t+1})$).
This results in a new state $s_{t+1} = (P_{t+1}, x_{t+1})$.


### The reward function
SeaPearl uses a "propagation-based reward". As the goal of the agent is to quickly find a good solution, it needs to learn to effectively prune the search space and move toward promising regions. An intuitive way to configure the reward is to give the agent the objective value, but this information is only available at the end of episodes. In other words, it makes the reward signal extremely sparse. To address this problem, SeaPearl uses both an intermediate reward and a final reward. The intuition behind the intermediate reward is this: it is computed by rewarding the pruning of high values from the variable's domain and penalizing the pruning of low values from the variable's domain. Mathematcially, the intermediate reward is defined as follows:

$$

r_t^{ub} = \{ v \in D_t(x_{\text{{obj}}}) \mid v \notin D_{t+1}(x_{\text{{obj}}}) \land v > \max(D_t(x_{\text{{obj}}})) \} \\
r_t^{lb} = \{ v \in D_t(x_{\text{{obj}}}) \mid v \notin D_{t+1}(x_{\text{{obj}}}) \land v < \min(D_t(x_{\text{{obj}}})) \} \\
r^{mid}_t = \frac{{r_t^{ub} - r_t^{lb}}}{{\lvert D_1(x_{\text{{obj}}}) \rvert}} \\
r^{end}_t = \begin{cases} -1 & \text{{if unfeasible solution found}} \\ 0 & \text{{otherwise}} \end{cases} \\
r_{acc} = \frac{{\sum_{t=1}^{T} (r^{mid}_t + r^{end}_t)}}{{T-1}}
$$

## Implementation

We will now begin to implement the MIS problem in SeaPearl. To start, we will define the reward, which comes directly from the mathematical definition above. It is implemented in the `GeneralReward` of SeaPearl.


In [3]:
reward = SeaPearl.GeneralReward

SeaPearl.GeneralReward

# The Neural Network

Next up, we need to define the neural network that will learn the value selection function. As the problems can differ in size, the use of graph neural networks (GNNs) is particularly appropriate. GNNs are a class of neural networks that operate on graphs, which means we need to convert the problem instances to graphs. In SeaPearl, we use tripartite graphs, which are graphs with three types of nodes: variables, values and constraints. There is one node for every variable, for every value and for every constraint. The edges are defined as follows:
 - There is an edge between a variable and a value if the value is in the domain of the variable.
 - There is an edge between a variable and a constraint if the variable appears in the constraint.

Nodes have the following features:
 - Values have a one-hot encoding of their value. For example, if the domain of a variable is $\{1, 2, 3\}$, then the values will have a one-hot encoding of $[1, 2, 3]$.
 - Constraints have a one-hot encoding of their type (i.e., constraint).

Other features can be used in the graph and we will define a featurization for it later on.

## Setting up the experiment

In the next cell, we will set up the experiment. We will create structs for the agent and the experiment. We will also define the hyperparameters of the experiment.

In [4]:
"""MisAgentConfig holds parameters for the configuration of the RL agent that will be used"""

struct MisAgentConfig
    gamma::Float32  # Discount factor for future rewards
    batch_size::Int  # Batch size
    output_size::Int  # One for every node
    update_horizon::Int  # How many steps to look ahead in the environment
    min_replay_history::Int  # Minimum number of transitions to keep in the replay buffer
    update_freq::Int  # Number of actions between successive SGD updates
    target_update_freq::Int  # Number of actions between target network updates
    trajectory_capacity::Int  # Maximum number of trajectories to store in the replay buffer
end

"""MisExperimentSettings holds parameters for the configuration of the experiment"""
struct MisExperimentSettings
    nbEpisodes::Int # Number of episodes in the training
    restartPerInstances::Int # Number of restarts per instance during training
    evalFreq::Int # Number of episodes between evaluations; an evalFreq of 20 means that the performance of heuristics -the agent and the other ones, will be evaluated every 20 episodes
    nbInstances::Int # Number of instances to train on
    nbRandomHeuristics::Int # Number of random heuristics to use for comparison purposes
    nbNewVertices::Int 
    nbInitialVertices::Int
    seedEval::Int # Random seed
end

agent_config = MisAgentConfig(0.99f0, 64, instance_generator.n, 4, 400, 1, 20, 2000)
mis_settings = MisExperimentSettings(300, 1, 10, 10, 1, numNewVertices, numInitialVertices, 123)

MisExperimentSettings(300, 1, 10, 10, 1, 8, 3, 123)

## Further configuration

Next up, we will define additional configurations for the experiment:
- The random seed
- Number of steps per episode
- The update horizon
- The device to use (CPU or GPU)
- The evaluation frequency
- The steps for the explorer
- The parameter initialization function

In [5]:
n_step_per_episode = Int(round(mis_settings.nbNewVertices // 2)) + mis_settings.nbInitialVertices 
update_horizon = Int(round(n_step_per_episode // 2))
device = cpu # change if you have a GPU

if device == gpu
    CUDA.device!(numDevice)
end

evalFreq = mis_settings.evalFreq # evaluation frequency, as defined above
decay_steps_explorer = Int(floor(mis_settings.nbEpisodes * n_step_per_episode / 2)) # Number of steps after which the explorer will perform exponential decay for epsilon
generator = instance_generator # Use the 
eval_generator = generator

rngExp = MersenneTwister(mis_settings.seedEval)
init = Flux.glorot_uniform(MersenneTwister(mis_settings.seedEval))

#1 (generic function with 1 method)

## The State Representation

The state representation is defined in the `HeterogeneousStateRepresentation` class. It is a heterogeneous state representation, which means that it is comprised of multiple state representations. In this example, we will use the `DefaultFeaturization` and the `HeterogeneousTrajectoryState` state representations. The `DefaultFeaturization` state representation is a featurization that is used by default in SeaPearl and allows the user to select the graph features they want. The available features are the following.
### Variable Features:
- node_number_of_neighbors
- variable_initial_domain_size
- variable_domain_size
- variable_is_bound
- variable_is_branchable
- variable_is_objective
- variable_assigned_value
### Constraint Features
- node_number_of_neighbors
- constraint_activity
- nb_involved_constraint_propagation
- nb_not_bounded_variable
- constraint_type
### Value Features
- node_number_of_neighbors
- values_raw
- values_onehot

The featurization is used to convert the problem instance to a graph. The `HeterogeneousTrajectoryState` state representation is a state representation that is used to represent the state at a given point in the resolution of the problem. It contains the variable that is branched on, the feature graph at that point and the available values.

### Visual representation

The following image shows an example of a state representation. Here:
- $f_1$ represents features attached to variables ($X_1, X_2, X_3$)
- $f_2$ represents features attached to constraints ($C_1, C_2, C_3$)
- $f_3$ represents features attached to values (1, 2, 3)

<img src="img/state-representation.png" alt="State Representation" style="width:700px; height:250px;">

In [6]:
# Defines the features that will be used
chosen_features = Dict(
    "node_number_of_neighbors" => true,
    "constraint_type" => true,
    "constraint_activity" => true,
    "nb_not_bounded_variable" => true,
    "variable_initial_domain_size" => true,
    "variable_domain_size" => true,
    "variable_is_objective" => true,
    "variable_assigned_value" => true,
    "variable_is_bound" => true,
    "values_raw" => true
)
SR_heterogeneous = SeaPearl.HeterogeneousStateRepresentation{SeaPearl.DefaultFeaturization,SeaPearl.HeterogeneousTrajectoryState}

SeaPearl.HeterogeneousStateRepresentation{SeaPearl.DefaultFeaturization, SeaPearl.HeterogeneousTrajectoryState}

## Defining the Neural Network

In the next cell, we will define the neural network used by the RL agent. We will be using Graph Neural Networks (GNNs) as the learnable architecture. The inputs will be graphs as defined earlier. The features are defined in a Dict and contain elements coming from the problem instance.

## The GNN model

The model used in this example is drawn from [Marty et al. 2023](https://arxiv.org/abs/2301.01913). The model is defined as a heterogeneous GNN, meaning that every node type uses a specific convolution. The model is comprised of the following elements:

### GNN encoder
The update equations are given by the following:
\begin{align*}
h_{x}^{k+1} &= g\left(\theta_{1}^{k}h_{x}^{0} || \theta_{2}^{k}h_{x}^{k} || \oplus_{c\in N_{c}(x)}\theta_{3}^{k}h_{c}^{k} || \oplus_{v\in N_{v}(x)}\theta_{4}^{k}h_{v}^{k}\right) \quad &\forall x \in V_1 \\
h_{c}^{k+1} &= g\left(\theta_{5}^{k}h_{c}^{0} || \theta_{6}^{k}h_{c}^{k} || \oplus_{x\in N_{x}(c)}\theta_{7}^{k}h_{x}^{k}\right) \quad &\forall c \in V_2 \\
h_{v}^{k+1} &= g\left(\theta_{8}^{k}h_{v}^{0} || \theta_{9}^{k}h_{v}^{k} || \oplus_{x\in N_{x}(v)}\theta_{10}^{k}h_{x}^{k}\right) \quad &\forall v \in V_3
\end{align*}

- $h_{x}^{k}$, $h_{c}^{k}$, $h_{v}^{k}$ are the feature vectors for nodes of type variable, constraint, and value at layer $k$ respectively.
- $h_{x}^{0}$, $h_{c}^{0}$, $h_{v}^{0}$ are the feature vectors for nodes of type variable, constraint, and value from the previous layer. They represent the skip-connections.
- $g$ is the leakyReLU activation function which introduces non-linearity into the model.
- $\oplus$ is the mean aggregation function.
- $||$ is the concatenation operator.
- $\theta_{i}^{k}$ are weight matrices at layer $k$ which are learned during training.
- $N_{c}$, $N_{v}$ and $N_{x}$ are the sets of neighboring nodes for each node.

### The action decoder
Actions are selected using an epsilon-greedy policy. The Q-values are computed like this:

$$\widehat Q(h_{x}^{K}, h_{v}^{K}) = \phi_{q}(\phi_{x}(h_{x}^{K}) \oplus \phi_{v}(h_{v}^{K})) \quad \forall v \in V_{x}$$

- $h_{x}^{K}$ and $h_{v}^{K}$ are the final node embeddings for the variable and value nodes, respectively, after $K$ iterations in the GNN.
- $\phi_{x}$ and $\phi_{v}$ are fully connected neural networks that take as input the embeddings of the variable and value nodes, respectively. They further transform these embeddings into an intermediate representation.
- $\oplus$ denotes concatenation of vectors.
- $\phi_{q}$ is another fully connected neural network that takes the concatenated vector as input and outputs a single scalar value, the Q-value.
- $V_{x}$ is the subset of value nodes that can be assigned to variable $x$.


### Visual Representation
 
Putting it all together, we get the following model:

<img src="img/High-level-architecture.png" alt="Architecture" style="width:900px; height:300px;">


In [7]:
"""Struct that holds the layers of the neural network"""
struct HeterogeneousModel{A,B}
    Inputlayer::A
    Middlelayers::Vector{B}
end

Flux.@functor HeterogeneousModel # To allow automatic differentiation
"""Passes feature graphs (instances) through the neural network, by always passing the original feature graph
as well as the current feature graph to the layers of the network. 
"""
function (m::HeterogeneousModel)(fg)
    original_fg = deepcopy(fg)
    out = m.Inputlayer(fg)
    for layer in m.Middlelayers
        out = layer(out, original_fg)
    end
    return out
end

# The size of the input features for each type of node (variable, constraint, value), respectively
feature_size = [6, 5, 2]

"""
    get_dense_chain(in, mid, out, n_layers, σ=Flux.identity; init=Flux.glorot_uniform)

Create a chain of dense layers for a neural network.

# Arguments
- `in::Int`: The size of the input layer.
- `mid::Int`: The size of the intermediate layers.
- `out::Int`: The size of the output layer.
- `n_layers::Int`: The number of layers in the chain.
- `σ::Function=Flux.identity`: The activation function to use.
- `init::Function=Flux.glorot_uniform`: The initialization method to use.

# Returns
A `Flux.Chain` object representing the chain of dense layers.

# Examples
```julia
julia> get_dense_chain(10, 20, 5, 3)
Chain(Dense(10, 20, σ), Dense(20, 20, σ), Dense(20, 5))
```
"""
function get_dense_chain(in, mid, out, n_layers, σ=Flux.identity; init=Flux.glorot_uniform)
    @assert n_layers >= 1
    layers = []
    if n_layers == 1
        push!(layers, Flux.Dense(in, out, init=init))
    elseif n_layers == 2
        push!(layers, Flux.Dense(in, mid, σ, init=init))
        push!(layers, Flux.Dense(mid, out, init=init))
    else
        push!(layers, Flux.Dense(in, mid, σ, init=init))
        for i in 2:(n_layers-1)
            push!(layers, Flux.Dense(mid, mid, σ, init=init))
        end
        push!(layers, Flux.Dense(mid, out, init=init))
    end
    return Flux.Chain(layers...)
end

# Builds the SeaPearl HeterogeneousFullFeaturedCPNN model
function build_model(; 
        feature_size,
        conv_size=8,
        dense_size=16,
        output_size=1,
        n_layers_graph=3,
        n_layers_node=2,
        n_layers_output=2,
        pool=SeaPearl.meanPooling(),
        σ=Flux.leakyrelu,
        init=Flux.glorot_uniform,
        device=cpu
    )
    input_layer = SeaPearl.HeterogeneousGraphConvInit(feature_size, conv_size, σ, init=init) # input layer
    middle_layers = SeaPearl.HeterogeneousGraphConv[] # middle layers
    for i in 1:n_layers_graph-1
        push!(middle_layers, SeaPearl.HeterogeneousGraphConv(conv_size => conv_size, feature_size, σ, pool=pool, init=init))
    end
    output_layer = SeaPearl.HeterogeneousGraphConv(conv_size => output_size, feature_size, σ, pool=pool, init=init) # output layer
    dense_layers = get_dense_chain(conv_size, dense_size, dense_size, n_layers_node, σ, init=init) # dense layers
    # Define the final output layer
    final_output_layer = get_dense_chain(2 * dense_size, dense_size, output_size, n_layers_output, σ, init=init)

    # Build the model
    model = SeaPearl.HeterogeneousFullFeaturedCPNN(
        HeterogeneousModel(input_layer, middle_layers),
        dense_layers,
        Flux.Chain(),
        final_output_layer
    ) |> device

    return model
end

build_model (generic function with 1 method)

## The Agent, Replay Buffer and Exploration Policy

We now have:
- The reward function
- An instance generator
- A GNN
- And all the settings we need!

We now need to define:
- The way we will store trajectories will be stored (in a circular buffer)
- The exploration policy (eps-greedy)
- The agent (DQN)

In [8]:
"""
    get_heterogeneous_slart_trajectory(; capacity, n_actions)

Create a circular buffer for storing trajectories in the context of reinforcement learning where not all actions are legal. 
SLART stands for State, Legal Actions, Reward, Terminal.

# Arguments
- `capacity::Int`: The maximum number of trajectories that can be stored in the buffer.
- `n_actions::Int`: The number of possible actions that can be taken at each time step.

# Returns
A `CircularArraySLARTTrajectory` object with the specified capacity and legal actions mask, and an empty state buffer.
"""
function get_heterogeneous_slart_trajectory(; capacity, n_actions)
    return RL.CircularArraySLARTTrajectory(
        capacity=capacity,
        state=SeaPearl.HeterogeneousTrajectoryState[] => (),
        legal_actions_mask=Vector{Bool} => (n_actions,),
    )
end

"""Builds the DQN agent for the heterogeneous model"""
function get_heterogeneous_agent(; get_explorer, batch_size=16, update_horizon, min_replay_history, update_freq=1, target_update_freq=200, γ=0.999f0, get_heterogeneous_trajectory, get_heterogeneous_nn)
    return RL.Agent(
        policy=RL.QBasedPolicy(
            learner=get_heterogeneous_learner(batch_size, update_horizon, min_replay_history, update_freq, target_update_freq, get_heterogeneous_nn, γ),
            explorer=get_explorer(),
        ),
        trajectory=get_heterogeneous_trajectory()
    )
end

"""Builds the learning part of the DQN agent"""
function get_heterogeneous_learner(batch_size, update_horizon, min_replay_history, update_freq, target_update_freq, get_heterogeneous_nn, γ)
    return RL.DQNLearner(
        approximator=RL.NeuralNetworkApproximator(
            model=get_heterogeneous_nn(),
            optimizer=ADAM()
        ),
        target_approximator=RL.NeuralNetworkApproximator(
            model=get_heterogeneous_nn(),
            optimizer=ADAM()
        ),
        loss_func=Flux.Losses.huber_loss,
        batch_size=batch_size,
        update_horizon=update_horizon,
        min_replay_history=min_replay_history,
        update_freq=update_freq,
        target_update_freq=target_update_freq,
        γ=γ
    )
end

"""
    get_epsilon_greedy_explorer(decay_steps, ϵ_stable; rng=nothing)

Create an epsilon-greedy explorer for use in reinforcement learning.

# Arguments
- `decay_steps::Int`: The number of steps over which to decay the exploration rate.
- `ϵ_stable::Real`: The minimum exploration rate to use after decay.
- `rng::AbstractRNG`: (optional) A random number generator to use for sampling actions.

# Returns
An `EpsilonGreedyExplorer` object with the specified exploration rate decay and random number generator.
"""
function get_epsilon_greedy_explorer(decay_steps, ϵ_stable; rng=nothing)
    if isnothing(rng)
        return RL.EpsilonGreedyExplorer(
            ϵ_stable=ϵ_stable,
            kind=:exp,
            decay_steps=decay_steps,
            step=1
        )
    else
        return RL.EpsilonGreedyExplorer(
            ϵ_stable=ϵ_stable,
            kind=:exp,
            decay_steps=decay_steps,
            step=1,
            rng=rng
        )
    end
end


get_epsilon_greedy_explorer

## Agent definition

The agent and the related learned heuristic are defined here.

In [ ]:
pool = SeaPearl.meanPooling()

agent = get_heterogeneous_agent(;
    get_heterogeneous_trajectory=() -> get_heterogeneous_slart_trajectory(capacity=agent_config.trajectory_capacity, n_actions=2),
    get_explorer=() -> get_epsilon_greedy_explorer(decay_steps_explorer, 0.01; rng=rngExp),
    batch_size=agent_config.batch_size,
    update_horizon=update_horizon,
    min_replay_history=Int(round(16 * n_step_per_episode // 2)),
    update_freq=agent_config.update_freq,
    target_update_freq=agent_config.target_update_freq,
    get_heterogeneous_nn=() -> build_model(
        feature_size=feature_size,
        conv_size=8,
        dense_size=16,
        output_size=1,
        n_layers_graph=3,
        n_layers_node=3,
        n_layers_output=2,
        pool=pool,
        σ=NNlib.leakyrelu,
        init=init,
        device=device
    ),
    γ=0.99f0
)

## Setting up comparisons and running the experiment

We now have everything we need to run the experiment. We will run the experiment for 1000 episodes and compare the performance of value selection heuristics:
- The learned heuristic
- A random agent 
- Heuristic that always selects the max value available.

The agent has already been defined; let's define the other two heuristics.

In [ ]:
"""Selects the maximum value from the domain of a variable"""
selectMax(x::SeaPearl.IntVar; cpmodel=nothing) = SeaPearl.maximum(x.domain)

"""Selects a random value from the domain of a variable"""
function select_random_value(x::SeaPearl.IntVar; cpmodel=nothing)
    selected_number = rand(1:length(x.domain))
    i = 1
    for value in x.domain
        if i == selected_number
            return value
        end
        i += 1
    end
    @assert false "This should not happen"
end

randomHeuristics = []
for i in 1:mis_settings.nbRandomHeuristics
    push!(randomHeuristics, SeaPearl.BasicHeuristic(select_random_value))
end

heuristic_max = SeaPearl.BasicHeuristic(selectMax)
learned_heuristic = SeaPearl.SimpleLearnedHeuristic{SR_heterogeneous,reward,SeaPearl.FixedOutput}(agent; chosen_features=chosen_features)
valueSelectionArray = [learned_heuristic, heuristic_max]
append!(valueSelectionArray, randomHeuristics)

## Variable selection heuristic

For all experiments, we will be using the minimum domain heuristic variable selection heuristic. This heuristic selects the variable with the smallest domain size.

In [11]:
variableSelection = SeaPearl.MinDomainVariableSelection{false}()

SeaPearl.MinDomainVariableSelection{false}()

## Solving the problem

Let's finally solve the problem. The following cell will train a DQN agent on MIS instances and evaluate its performance on the same instances.

In [ ]:
metricsArray, eval_metricsArray = SeaPearl.train!(
    valueSelectionArray=valueSelectionArray,
    generator=instance_generator,
    nbEpisodes=mis_settings.nbEpisodes,
    strategy=SeaPearl.DFSearch(),
    variableHeuristic=variableSelection,
    out_solver=true,
    verbose=false,
    evaluator=SeaPearl.SameInstancesEvaluator(valueSelectionArray, instance_generator; evalFreq=mis_settings.evalFreq, nbInstances=mis_settings.nbInstances),
    restartPerInstances=mis_settings.restartPerInstances
)

## Visualizing the results

Now that we have a small trained model, let's look at the results. First, we will have a look at the performance of the learned heuristic vs random and select max heuristic over the course of the training. Then, we will compare their performance on unseen instances of the same size.

In [32]:
# Utility functions fetch and organize training metrics + plotting & benchmarking
include("../../../learning_cp/utils/save_metrics.jl")
include("../../../learning_cp/utils/plot_metrics.jl")
include("../../../learning_cp/utils/benchmark.jl")

benchmark (generic function with 1 method)

## Training performance

In this section, we will be comparing the performance of the learned heuristic with the random and select max heuristics. We will be looking at the performance of the heuristics over the course of the training. The plot in the next cell represents the optimum obtained by the heuristics at every evaluation step.

In [ ]:
training_df = get_metrics_dataframe(eval_metricsArray)
plot_first_solution(training_df, 0, 5)

## Evaluation on unseen instances

Next up, we will compare the performances of the heuristics on unseen instances. In this section, you are free to play with the various parameters and see their effect. The plot in this section represents the optimum obtained by the heuristics for the first solution they find. For small instances, you should see that the learned heuristic learns something close to the select max heuristic.

In [ ]:
using DataFrames, Plots

validation_generator = SeaPearl.MaximumIndependentSetGenerator(8, 3)

num_instances = 20 # Number of instances to evaluate on
node_budget = 10000 # Budget of visited nodes
take_objective = false # Set it to true if we have to branch on the object ive variable
eval_strategy = SeaPearl.DFSearch() #
include_dfs = true # Set it to true if you want to evaluate with DFS in addition to ILDS
basicHeuristics = Dict()
num_random_heuristics = 2

for (i, random_heuristic) in enumerate(randomHeuristics)
    push!(basicHeuristics, "random"*string(i) => random_heuristic)
end

push!(basicHeuristics, "max" => heuristic_max)
evaluation_df = benchmark(
    models=[agent.policy.learner.approximator], 
    evaluation_folder=pwd(),
    num_instances=num_instances, 
    chosen_features=chosen_features,
    take_objective=take_objective,
    generator=validation_generator,
    basicHeuristics=basicHeuristics,
    save_experiment_metrics=false,
    include_dfs=include_dfs, 
    budget=node_budget,
    ILDS=eval_strategy
)

## Comparing heuristics across 20 instances

We will now compare the performance of the learned heuristic with the random and select max heuristics on 20 unseen instances of the same size.

In [ ]:
# Filter the evaluation_df to include only the SimpleLearnedHeuristic
learned_heuristic_df = filter(row -> row[:heuristic_type] == "SimpleLearnedHeuristic", evaluation_df)
# Filter the evaluation_df to include only the BasicHeuristic(selectMax)
select_max_df = filter(row -> row[:heuristic_type] == "BasicHeuristic(selectMax)", evaluation_df)
# Filter the evaluation_df to include only the BasicHeuristic(select_random_value)
random_heuristic_df = filter(row -> row[:heuristic_type] == "BasicHeuristic(select_random_value)", evaluation_df)

# Create a line plot of the first solution for each instance for all heuristics
scatter()
scatter!(learned_heuristic_df[!, :num_instance], -learned_heuristic_df[!, :first_sol], label="SimpleLearnedHeuristic", markersize=8)
scatter!(select_max_df[!, :num_instance], -select_max_df[!, :first_sol], label="selectMax")
scatter!(random_heuristic_df[!, :num_instance], -random_heuristic_df[!, :first_sol], label="select_random_value")
xlabel!("Instance Number")
ylabel!("First Solution")
xticks!(1:maximum(evaluation_df[!, :num_instance]), visible=true)

## Solve Large Problems with Pre-Trained Model

The last example was on trivially small problems. Now let's have a look at the performance of learned heuristics on larger problems. For this section, we will load a model trained on MIS problems with 40 nodes and we will be comparing the performance of the learned heuristc with random heuristics and the select max heuristic.

In [ ]:
validation_generator = SeaPearl.MaximumIndependentSetGenerator(40, 4)
num_instances = 20 # Number of instances to evaluate on
node_budget = 10000 # Budget of visited nodes
take_objective = false # Set it to true if we have to branch on the object ive variable
eval_strategy = SeaPearl.DFSearch()
include_dfs = true # Set it to true if you want to evaluate with DFS in addition to ILDS
basicHeuristics = Dict()
num_random_heuristics = 2

for (i, random_heuristic) in enumerate(randomHeuristics)
    push!(basicHeuristics, "random" * string(i) => random_heuristic)
end

folder = "./models"
models = []
for file in readdir(folder)
    if splitext(file)[2] == ".bson"
        println(folder * "/" * file)
        @load folder * "/" * file model
        push!(models, model)
    end
end

for (i, random_heuristic) in enumerate(randomHeuristics)
    push!(basicHeuristics, "random" * string(i) => random_heuristic)
end

push!(basicHeuristics, "max" => heuristic_max)
evaluation_df = benchmark(
    models=models,
    evaluation_folder=pwd(),
    num_instances=num_instances,
    chosen_features=chosen_features,
    take_objective=take_objective,
    generator=validation_generator,
    basicHeuristics=basicHeuristics,
    save_experiment_metrics=false,
    include_dfs=include_dfs,
    budget=node_budget,
    ILDS=eval_strategy
)

## Looking at results

We have now loaded a model trained on MIS problems with 40 nodes. Let's have a look at the performance of the learned heuristic vs random and select max heuristic over the course of the training. 

In [ ]:
# Filter the evaluation_df to include only the SimpleLearnedHeuristic
learned_heuristic_df = filter(row -> row[:heuristic_type] == "SimpleLearnedHeuristic", evaluation_df)
# Filter the evaluation_df to include only the BasicHeuristic(selectMax)
select_max_df = filter(row -> row[:heuristic_type] == "BasicHeuristic(selectMax)", evaluation_df)
# Filter the evaluation_df to include only the BasicHeuristic(select_random_value)
random_heuristic_df = filter(row -> row[:heuristic_type] == "BasicHeuristic(select_random_value)", evaluation_df)

# Create a line plot of the first solution for each instance for all heuristics
scatter()
scatter!(learned_heuristic_df[!, :num_instance], -learned_heuristic_df[!, :first_sol], label="SimpleLearnedHeuristic", markersize=8)
scatter!(select_max_df[!, :num_instance], -select_max_df[!, :first_sol], label="selectMax")
scatter!(random_heuristic_df[!, :num_instance], -random_heuristic_df[!, :first_sol], label="select_random_value")
xlabel!("Instance Number")
ylabel!("First Solution")
xticks!(1:maximum(evaluation_df[!, :num_instance]), visible=true)